# 한국어 언어모델 학습 및 다중 과제 튜닝
## KoGPT-2 기반의 챗봇 실습

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
1기 멘토
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
2기 멘토
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
이녕우 (leenw2@gmail.com)
박채훈 (qkrcogns2222@gmail.com)
3기 멘토
이녕우 (leenw2@gmail.com)
박채훈 (qkrcogns2222@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

In [2]:
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Updated git hooks.
Git LFS initialized.
Cloning into 'kogpt2'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 52 (delta 20), reused 52 (delta 20), pack-reused 0
Unpacking objects: 100% (52/52), done.
Filtering content: 100% (2/2), 959.93 MiB | 32.61 MiB/s, done.


## 데이터셋 준비

In [ ]:
!git clone https://github.com/songys/Chatbot_data.git

## 라이브러리 설치

In [ ]:
%pip install transformers

In [6]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel
import pandas as pd
from transformers import AdamW
from torch.utils.data import DataLoader
torch.manual_seed(42)

In [8]:
tokenizer = SentencePieceBPETokenizer("./kogpt2/vocab.json", "./kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = './kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [9]:
data = pd.read_csv('./Chatbot_data/ChatbotData.csv')

In [10]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])  # special token을 몇 개나 추가했는지 파악해서 꼭 모델을 리사이즈해줘야 함
print(added_special_token_num)

0


In [11]:
pad_id = tokenizer.token_to_id("<pad>")
print(pad_id)
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>")
tokenizer.enable_truncation(max_length=128)

3


In [12]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer
    
    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>'
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data)
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        return item

In [13]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='./Chatbot_data/ChatbotData.csv')
train_dataset.load_data()
data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')
        
        outputs = model(data, labels=data)
        loss, logits = outputs[:2]
        loss = loss.to('cuda')
        loss.backward()
        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1


In [15]:
torch.save(model.state_dict(), 'chitchat_model.pth')

In [28]:
tokenizer.save_pretrained("mytokenizer")

AttributeError: 'SentencePieceBPETokenizer' object has no attribute 'save_pretrained'

In [16]:
def encoding(text):
    text = '<s>'+text+'</s><s>'
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [19]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=0.95, 
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]]  # 생성 단어로 unk토큰이 선택될 경우 다른 단어를 선택하도록 설정
    )

    decoded_result = decoding(sample_outputs.tolist())
    return decoded_result

In [20]:
decoded_result = get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')
print(type(decoded_result))

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


<class 'list'>


In [27]:
decoded_result[4]

'인공지능의 미래에 대해 어떻게 생각하세요? 인공지능의 미래는 어떻게 바뀔까요. 인공지능에는 관심있는 사람이 있을거라 생각해요. 인공지능에 인공지능은 중요한 변화가 있을지도 모르니까요.'

In [18]:
get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 미래의 가장 중요한 사람이 될 거예요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능의 미래는 정해져있지 않아요. 인공지능의 미래를 함께 고민해보는 건 어떨까요. 인공지능의 미래에 대한 고민이신가요?
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능에 물음을 걸어보세요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 만든 미래엔 인공지능이 있죠. 인공지능의 미래는 로봇이지요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 수많은 생각을 하게 만들죠. 인공지능의 미래를 걱정하고 있나요. 인공지능을 두려워하지 마세요.
